# TP 3 : Paas le TP2


relation de récurrence : C_i,j = max{C_i−1,j, C_i−1,j−wi + c_i}

### Initialisation (à faire une seule fois)

In [ ]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

### Récupération des données

In [15]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

In [84]:
function solveKnapInstance(filename)
    price, weight, capacity = readKnaptxtInstance(filename)
    Bestsol=Float64[]

    function C(i, j)
        if j <= 0 || i <= 0
            return 0
        else
            c1 = C(i-1, j)
            c2 = C(i-1, j-weight[i]) + price[i]
            if j-weight[i] < 0
                return c1
            else
                return max(c1, c2)
            end
        end
    end

    BestProfit = C(length(weight), capacity)

    i = length(weight)
    j = capacity
    while i > 1
        c1 = C(i-1,j)
        c2 = C(i-1,j-weight[i]) + price[i]
        if c1 > c2
            j = j
            i = i-1
        else
            push!(Bestsol, i)
            j = j-weight[i]
            i = i-1
        end
    end

    print("\t")
    for j in 0:capacity
        print("j=", j, "\t")
    end
    print("\n")
    for i in 1:(length(weight))
        print("i=", i, "\t")
        for j in 0:capacity
            print(C(i,j), "\t")
        end
        print("\n")
    end
    
    
    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol

end


solveKnapInstance (generic function with 1 method)

In [85]:
INSTANCE = "InstancesKnapSack/test.opb.txt"

solveKnapInstance(INSTANCE)

	j=0	j=1	j=2	j=3	j=4	j=5	j=6	j=7	j=8	j=9	j=10	
i=1	0	0	0	0	0	0	0	42	42	42	42	
i=2	0	0	0	0	40	40	40	42	42	42	42	
i=3	0	0	0	12	40	40	40	52	52	52	54	
i=4	0	0	0	12	40	40	40	52	52	65	65	

******

Optimal value = 65

Optimal x=[4.0, 2.0]


(65, [4.0, 2.0])